In [1]:
! nvidia-smi



Thu Nov  2 14:22:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00


In [3]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [4]:
model = 'SkAndMl/english-to-hinglish'

In [5]:
dataset = load_dataset("findnitai/english-to-hinglish")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 189102
    })
})

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [8]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["kya isane theaters mein achchha pradarshan kiya?"]))

{'input_ids': [[6984, 667, 292, 23, 980, 393, 4, 2665, 1809, 44, 5049, 568, 19, 3826, 3826, 6273, 3914, 4874, 232, 2905, 16, 13882, 59352, 667, 292, 22, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [9]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi_ng"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
preprocess_function(dataset["train"][:2])

{'input_ids': [[172, 70, 16, 4, 300, 8, 4, 16545, 0], [10595, 2, 4, 22802, 51898, 9880, 23, 475, 138, 4, 18200, 43347, 9880, 3803, 19, 978, 4556, 19, 16545, 8, 90, 3425, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[4135, 3022, 363, 6984, 292, 6984, 667, 292, 6532, 292, 292, 363, 13421, 624, 0], [6532, 16686, 16, 5081, 2, 44, 24694, 7049, 49668, 292, 7, 363, 2665, 818, 1411, 44, 22722, 3383, 3771, 292, 980, 13421, 624, 2, 44, 3713, 12431, 3383, 5606, 292, 3637, 896, 624, 7583, 44, 22722, 23, 3077, 2930, 20991, 142, 142, 292, 6984, 393, 393, 3914, 6887, 1279, 363, 44, 16, 393, 5680, 6716, 7049, 6984, 292, 363, 44, 3813, 1328, 2665, 292, 13421, 624, 3, 0]]}

In [11]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/189102 [00:00<?, ? examples/s]

In [12]:
model1 = TFAutoModelForSeq2SeqLM.from_pretrained(model,from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMarianMTModel: ['lm_head.weight']
- This IS expected if you are initializing TFMarianMTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMarianMTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFMarianMTModel were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 3

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [15]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [16]:
train_dataset = model1.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)


In [17]:
generation_dataset = model1.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [18]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model1.compile(optimizer=optimizer)

In [19]:
model1.fit(train_dataset, validation_data=train_dataset, epochs=1)

11818/11818 [==============================] - 3957s 331ms/step - loss: 0.5256 - val_loss: 0.3914


In [20]:
model1.save_pretrained("model/")

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model)
model1 = TFAutoModelForSeq2SeqLM.from_pretrained("model/")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [24]:
input_text  = "I was waiting for my bag"

tokenized = tokenizer([input_text], return_tensors='np')
out = model1.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949  3383  9388    19  1565  4413   801   292  1328  6984   393 14454
    667   393  7210   292  1659  6984  2905    44  4874  6273  5680  6273
      0 61949]], shape=(1, 26), dtype=int32)


In [25]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

mai apne bag ke liye wait kar raha tha


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [28]:
input_text  = " So even if it's a big video, I will clearly mention all the products"

tokenized = tokenizer([input_text], return_tensors='np')
out = model1.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949   471    44  4454    44  1659    70    16    19   801   624  1328
     44 23730    44     2  3383  9388    44 24694  1220  6887    44  8906
    232  1182   897   142  2417    16  6984   393   801   292  2905   393
     44  5049   568   801   292   232   916  6984  2905    44  8959  2930
   1328   292    44     3    44     3    44     3    44     3    44     3
     44     3    44     3     0 61949 61949 61949 61949 61949 61949 61949
  61949 61949 61949 61949 61949 61949 61949 61949 61949]], shape=(1, 81), dtype=int32)


In [29]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

To if it's a big video, mai sabhi productions ke baare mein badal kar jaunga.......


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [30]:
input_text  = " Definitely share your feedback in the comment section"

tokenized = tokenizer([input_text], return_tensors='np')
out = model1.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949    44 24694  1220    16   393    19   292  1565  1851   393    44
     16  1182  1659  2417    44    16 29102  2417    44  5049   568 13687
   2905   393  6984  2905   393    44     3    44     3    44     3    44
      3    44     3    44     3    44     3    44     3    44     3    44
      3    44     3    44     3    44     3    44     3    44     3    44
      3    44     3    44     3    44     3    44     3    44     3    44
      3    44     3    44     3    44     3    44     3    44     3    44
      3    44     3    44     3    44     3    44     3    44     3    44
      3    44     3    44     3    44     3    44     3    44     3    44
      3    44     3    44     3    44     3    44     3    44     3    44
      3    44     3    44     3    44     3     0]], shape=(1, 128), dtype=int32)


In [31]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

sabse aapke suition section mein share kare.................................................
